In [167]:
import pandas as pd
import numpy as np
import os

Reference: https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c

In [168]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import when

In [169]:
spark = pyspark.sql.SparkSession.builder.master('local[2]').appName('H1B-3').getOrCreate()
training_data = spark.read.csv('../DATA/training_downsampling.csv', header = True, inferSchema = True)

Py4JError: An error occurred while calling o23.sessionState. Trace:
py4j.Py4JException: Method setConfString([class java.lang.String, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:748)



In [ ]:
test_data = spark.read.csv(
    '../DATA/test_downsampling.csv', header=True, inferSchema=True)


Py4JError: An error occurred while calling o24.read. Trace:
py4j.Py4JException: Method setConfString([class java.lang.String, class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:748)



In [ ]:
training_data = training_data.drop(col('_c0'))
test_data = test_data.drop(col('_c0'))


Py4JError: functions does not exist in the JVM

In [ ]:
cols = training_data.columns
cols.remove('EMPLOYER_NAME')

ValueError: Unable to parse datatype from schema. 'c'

In [ ]:
training_data = training_data.drop(col('EMPLOYER_NAME'))
training_data = training_data.withColumn("CASE_DURATION",col("CASE_DURATION").cast("int"))
training_data = training_data.withColumn("FULL_TIME_POSITION",col("FULL_TIME_POSITION").cast("int"))
test_data = test_data.drop(col('EMPLOYER_NAME'))
test_data = test_data.withColumn(
    "CASE_DURATION", col("CASE_DURATION").cast("int"))
test_data = test_data.withColumn(
    "FULL_TIME_POSITION", col("FULL_TIME_POSITION").cast("int"))


Py4JError: functions does not exist in the JVM

In [ ]:
from pyspark.ml.feature import VectorAssembler
try:
  cols.remove('CASE_STATUS')
except:
  pass
assembler = VectorAssembler(inputCols=cols,outputCol="features", handleInvalid='skip')


In [ ]:
len(np.unique(training_data.columns))

421

In [ ]:
training_data = assembler.transform(training_data)
test_data = assembler.transform(test_data)


KeyError: 'c'

In [ ]:
training_data.select("features").show()

Py4JError: functions does not exist in the JVM

In [ ]:
training_data = training_data.withColumn("CASE_STATUS",
                                         when(training_data.CASE_STATUS == 'CERTIFIED',1)
                                         .otherwise(when(training_data.CASE_STATUS == 'DENIED',0)
                                         .otherwise(when(training_data.CASE_STATUS == 'WITHDRAWN',3)
                                         .otherwise(2))))
test_data = test_data.withColumn("CASE_STATUS",
                                         when(test_data.CASE_STATUS ==
                                              'CERTIFIED', 1)
                                         .otherwise(when(test_data.CASE_STATUS == 'DENIED', 0)
                                                    .otherwise(when(test_data.CASE_STATUS == 'WITHDRAWN', 3)
                                                    .otherwise(2))))


ValueError: Unable to parse datatype from schema. 'p'

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(labelCol = 'CASE_STATUS',
                        featuresCol = 'features',
                        maxIter=1000, regParam=0, elasticNetParam=0).setFamily("multinomial")

KeyError: 'p'

In [ ]:
model = lr.fit(training_data)

Py4JError: An error occurred while calling o1340.getParam

In [ ]:
predict_train = model.transform(training_data)
predict_test = model.transform(test_data)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='CASE_STATUS', probabilityCol = 'probability', metricName='f1')
predict_test.printSchema()


root
 |-- AK: integer (nullable = true)
 |-- AL: integer (nullable = true)
 |-- AR: integer (nullable = true)
 |-- AZ: integer (nullable = true)
 |-- CA: integer (nullable = true)
 |-- CO: integer (nullable = true)
 |-- CT: integer (nullable = true)
 |-- DC: integer (nullable = true)
 |-- DE: integer (nullable = true)
 |-- FL: integer (nullable = true)
 |-- FM: integer (nullable = true)
 |-- GA: integer (nullable = true)
 |-- GU: integer (nullable = true)
 |-- HI: integer (nullable = true)
 |-- IA: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- IL: integer (nullable = true)
 |-- IN: integer (nullable = true)
 |-- KS: integer (nullable = true)
 |-- KY: integer (nullable = true)
 |-- LA: integer (nullable = true)
 |-- MA: integer (nullable = true)
 |-- MD: integer (nullable = true)
 |-- ME: integer (nullable = true)
 |-- MH: integer (nullable = true)
 |-- MI: integer (nullable = true)
 |-- MN: integer (nullable = true)
 |-- MO: integer (nullable = true)
 |-- MP: intege

In [ ]:
evaluator.evaluate(predict_test)

0.7823293065887243

In [ ]:
evaluator.evaluate(predict_train)

0.6124651897573412

In [ ]:
test_prediction = np.array(predict_test.select('prediction').collect())
train_prediction = np.array(predict_train.select('prediction').collect())
test_label = np.array(predict_test.select('CASE_STATUS').collect())
train_label = np.array(predict_train.select('CASE_STATUS').collect())


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
print(f1_score(test_label, test_prediction, average='macro'),
      f1_score(test_label, test_prediction, average='micro'))
print(balanced_accuracy_score(test_label, test_prediction),
      accuracy_score(test_label, test_prediction))


0.48449173844479493 0.768238128482061
0.6711894463512205 0.768238128482061


In [ ]:
print(f1_score(train_label, train_prediction, average='macro'),
      f1_score(train_label, train_prediction, average='micro'))
print(balanced_accuracy_score(train_label, train_prediction),
      accuracy_score(train_label, train_prediction))


0.6781219234356165 0.6749234763458566
0.6749237103563478 0.6749234763458566


In [ ]:
f1_score(train_label, train_prediction, average=None), f1_score(
    test_label, test_prediction, average=None)


(array([0.73204116, 0.63370757, 0.73902349, 0.60771548]),
 array([0.30689389, 0.86425794, 0.52573077, 0.24108435]))